In [1]:
import pandas as pd
import psycopg2
import pandas.io.sql as psql
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, KFold, cross_val_predict, cross_validate
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from statsmodels.distributions.empirical_distribution import ECDF
from sklearn.metrics import classification_report, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import balanced_accuracy_score, accuracy_score, roc_auc_score
from sklearn.metrics import plot_confusion_matrix, confusion_matrix
from collections import OrderedDict
from sklearn.manifold import TSNE
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import scale
from sklearn.decomposition import TruncatedSVD
import matplotlib.cm as cm
import time
import scipy.sparse
import seaborn as sns
import lightgbm as lgb
from math import radians, sin, cos, atan2, sqrt
import re
from sklearn.neighbors import BallTree
from sklearn.neighbors import KDTree
import pickle
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from shapely.geometry import Polygon, box
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from tensorflow.keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Concatenate, Input, Dropout, Bidirectional, concatenate
from tensorflow.keras.layers import Reshape, Flatten, MultiHeadAttention, Attention
from tensorflow.keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from keras.backend import clear_session
from sklearn.naive_bayes import MultinomialNB
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import emoji
from tensorflow.keras.optimizers import Adam
import os.path
import networkx as nx
from collections import Counter
from pecanpy import node2vec
from sklearn.feature_selection import mutual_info_classif, chi2, SelectKBest, SelectFromModel, SelectFpr, SelectFdr
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.sparse import hstack 
import stellargraph as sg
from io import BytesIO
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from keras.models import load_model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from nltk.tokenize import TweetTokenizer
from tensorflow.keras.optimizers.schedules import ExponentialDecay

%matplotlib inline

sns.set_theme()

In [2]:
unTPath = "../../../../../unT/ffunes/"
tmp_save = unTPath + "tmp_saves/"
figuresPath = unTPath + "figures/"

In [3]:
with open(tmp_save + 'multilayer_graph_city_twitter_us.pickle', 'rb') as handle:
    multilayer_graph_city = pickle.load(handle)  

In [4]:
with open(tmp_save + 'cities_twitter_us.pickle', 'rb') as handle:
    train_regions, val_regions, test_regions, encoder_t = pickle.load(handle)

train_regions_enc = encoder_t.transform(train_regions)
val_regions_enc = encoder_t.transform(val_regions)
test_regions_enc = encoder_t.transform(test_regions)
    
#cities = [train_cities, val_cities, test_cities, encoder_city]

#with open(tmp_save + 'cities_twitter_us.pickle', 'rb') as handle:
#    train_cities, val_cities, test_cities, encoder_city = pickle.load(handle)

train_na_shape = 429639
test_na_shape = 10000
val_na_shape = 9998

total_regions = len(np.unique(train_regions))

In [5]:
def create_rgcn_model(optimizer, num_classes, generator):
    clear_session()
    
    rgcn = sg.layer.RGCN(
        layer_sizes=[128, 64, num_classes],
        activations=["relu", "relu", "softmax"],
        generator=generator,
        num_bases=2,
        dropout=0.2,
        bias=True,
        #kernel_regularizer=tf.keras.regularizers.l2(1e-4)
    )
    
    x_inp, x_out = rgcn.in_out_tensors()
    
    #x_out = tf.keras.layers.Dense(units=num_classes, activation="softmax")(x_out)

    model = tf.keras.Model(inputs=x_inp, outputs=x_out)

    model.compile(optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

    return model

In [ ]:
train_idxs = np.arange(train_na_shape)
val_idxs = np.arange(train_na_shape, train_na_shape + val_na_shape)
test_idxs = np.arange(train_na_shape + val_na_shape, train_na_shape + val_na_shape + test_na_shape)

gcn_generator = sg.mapper.RelationalFullBatchNodeGenerator(multilayer_graph_city, weighted=True)

lr = ExponentialDecay(
        0.1,
        decay_steps=50,
        decay_rate=0.8,
        staircase=True
    )

clf = create_rgcn_model(
    Adam(learning_rate=lr),
    total_regions,
    gcn_generator
)

X_train, y_train = gcn_generator.flow(train_idxs, train_regions_enc)[0]
X_val, y_val = gcn_generator.flow(val_idxs, val_regions_enc)[0]

X_test = gcn_generator.flow(test_idxs)

clf.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=30, min_delta=0.0001, restore_best_weights=True),
    ],
    epochs=240000,
    shuffle=False    
)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/240000
